In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

# 데이터 로드
data = pd.read_csv('../data/pm25_pm10_merged.csv')

# 필요한 도시만 선택
cities = ['Beijing', 'Seoul', 'Tokyo']
data = data[data['City'].isin(cities)]

# 날짜를 datetime 형식으로 변환
data['Date'] = pd.to_datetime(data['Date'])

# 도시별로 데이터 피벗
data_pivoted = data.pivot(index='Date', columns='City', values='PM2.5 (µg/m³)')

# NaN 값을 0으로 대체
data_pivoted = data_pivoted.fillna(0)

# 중국(베이징)의 PM2.5 데이터를 feature로 사용
X = data_pivoted['Beijing'].values.reshape(-1, 1)

# 서울과 도쿄의 PM2.5 데이터를 target으로 사용
y_seoul = data_pivoted['Seoul'].values
y_tokyo = data_pivoted['Tokyo'].values

# PM2.5 수치를 등급으로 변환하는 함수
def pm25_to_grade(pm25):
    if pm25 <= 15:
        return '좋음'
    elif pm25 <= 35:
        return '보통'
    else:
        return '나쁨'

# 등급으로 변환
y_seoul_grade = np.array([pm25_to_grade(pm25) for pm25 in y_seoul])
y_tokyo_grade = np.array([pm25_to_grade(pm25) for pm25 in y_tokyo])

# 레이블 인코딩
le = LabelEncoder()
y_seoul_encoded = le.fit_transform(y_seoul_grade)
y_tokyo_encoded = le.fit_transform(y_tokyo_grade)

# 데이터 분할
X_train, X_test, y_seoul_train, y_seoul_test, y_tokyo_train, y_tokyo_test = train_test_split(
    X, y_seoul_encoded, y_tokyo_encoded, test_size=0.2, random_state=42)

# Gradient Boosting 모델 생성 및 학습 (서울)
gb_seoul = GradientBoostingClassifier(random_state=42)
gb_seoul.fit(X_train, y_seoul_train)

# Gradient Boosting 모델 생성 및 학습 (도쿄)
gb_tokyo = GradientBoostingClassifier(random_state=42)
gb_tokyo.fit(X_train, y_tokyo_train)

# 예측
y_seoul_pred = gb_seoul.predict(X_test)
y_tokyo_pred = gb_tokyo.predict(X_test)

# 결과 출력
print("서울 분류 결과:")
print(classification_report(y_seoul_test, y_seoul_pred, target_names=le.classes_))

print("\n도쿄 분류 결과:")
print(classification_report(y_tokyo_test, y_tokyo_pred, target_names=le.classes_))


서울 분류 결과:
              precision    recall  f1-score   support

          나쁨       0.70      0.69      0.70       272
          보통       0.51      0.70      0.59       235
          좋음       0.96      0.64      0.77       236

    accuracy                           0.67       743
   macro avg       0.73      0.67      0.68       743
weighted avg       0.72      0.67      0.68       743


도쿄 분류 결과:
              precision    recall  f1-score   support

          나쁨       0.33      0.05      0.09        92
          보통       0.55      0.78      0.64       306
          좋음       0.74      0.61      0.67       345

    accuracy                           0.62       743
   macro avg       0.54      0.48      0.47       743
weighted avg       0.61      0.62      0.59       743



In [ ]:
# !pip install plotly

   ---------------------------------------- 0.0/14.8 MB ? eta -:--:--
   -------------------------------------- - 14.2/14.8 MB 74.3 MB/s eta 0:00:01
   ---------------------------------------- 14.8/14.8 MB 54.9 MB/s eta 0:00:00


In [ ]:
# import pandas as pd
# import streamlit as st
# from sklearn.ensemble import GradientBoostingRegressor
# from sklearn.model_selection import train_test_split
# import plotly.express as px
# from sklearn.multioutput import MultiOutputRegressor

# # 데이터 로드 및 전처리
# @st.cache_data
# def load_data():
#     data = pd.read_csv("../data/pm25_pm10_merged.csv")  # 파일 경로 수정 필요
#     data['Date'] = pd.to_datetime(data['Date'])
#     return data

# # 모델 학습
# def train_model(data):
#     pivot_data = data.pivot(index='Date', columns='City', values='PM2.5 (µg/m³)').reset_index().fillna(0)
#     X = pivot_data[['Beijing']]
#     y = pivot_data[['Seoul', 'Tokyo', 'Delhi', 'Bangkok']]
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
#     base_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
#     model = MultiOutputRegressor(base_model)
#     model.fit(X_train, y_train)
#     return model, pivot_data

# # 예측 함수
# def predict_pm25(model, beijing_pm25):
#     input_value = [[beijing_pm25]]
#     predicted_pm25 = model.predict(input_value)
#     cities = ['Seoul', 'Tokyo', 'Delhi', 'Bangkok']
#     return dict(zip(cities, predicted_pm25[0]))

# # 도시 좌표 딕셔너리
# city_coords = {
#     'Seoul': (37.5665, 126.978),
#     'Tokyo': (35.6895, 139.6917),
#     'Beijing': (39.9042, 116.4074),
#     'Delhi': (28.7041, 77.1025),
#     'Bangkok': (13.7563, 100.5018)
# }

# # Streamlit 앱
# st.title("Beijing PM2.5 기반 도시별 미세먼지 예측 및 시간별 지도")

# # 데이터 로드
# data = load_data()
# model, pivot_data = train_model(data)

# # 탭 구성
# tab1, tab2 = st.tabs(["예측 지도", "시간별 데이터 지도"])

# # 탭 1: 예측 지도
# with tab1:
#     st.subheader("베이징 PM2.5 값을 입력해 예측")
#     beijing_pm25 = st.number_input("Beijing PM2.5 (µg/m³)", min_value=0.0, max_value=500.0, value=100.0, step=1.0)

#     if st.button("예측하기"):
#         predictions = predict_pm25(model, beijing_pm25)
#         predictions['Beijing'] = beijing_pm25  # 입력값 포함

#         # 예측 데이터프레임 생성
#         pred_df = pd.DataFrame({
#             'City': list(predictions.keys()),
#             'PM2.5 (µg/m³)': list(predictions.values()),
#             'Latitude': [city_coords[city][0] for city in predictions.keys()],
#             'Longitude': [city_coords[city][1] for city in predictions.keys()]
#         })

#         # 지도 시각화
#         fig = px.scatter_mapbox(pred_df, 
#                                 lat="Latitude", 
#                                 lon="Longitude", 
#                                 size="PM2.5 (µg/m³)", 
#                                 color="PM2.5 (µg/m³)", 
#                                 hover_name="City", 
#                                 hover_data={"PM2.5 (µg/m³)": True, "Latitude": False, "Longitude": False},
#                                 size_max=30,
#                                 zoom=2,
#                                 mapbox_style="open-street-map",
#                                 title=f"Beijing PM2.5 = {beijing_pm25} µg/m³일 때 예측")
#         st.plotly_chart(fig)

# # 탭 2: 시간별 데이터 지도 (슬라이더로 날짜 변경)
# with tab2:
#     st.subheader("시간별 미세먼지 농도 지도")
    
#     # 날짜 슬라이더
#     unique_dates = data['Date'].dt.date.unique()
#     min_date_idx = 0
#     max_date_idx = len(unique_dates) - 1
    
#     selected_idx = st.slider("날짜 선택 (막대바를 이동하세요)", 
#                              min_value=min_date_idx, 
#                              max_value=max_date_idx, 
#                              value=0, 
#                              format="")  # 숫자 대신 날짜로 표시하기 위해 format 비움
#     selected_date = unique_dates[selected_idx]
#     st.write(f"선택된 날짜: {selected_date}")

#     # 선택된 날짜 데이터 필터링
#     date_data = data[data['Date'].dt.date == selected_date].copy()
#     date_data['Latitude'] = date_data['City'].map(lambda x: city_coords[x][0])
#     date_data['Longitude'] = date_data['City'].map(lambda x: city_coords[x][1])

#     # 지도 시각화
#     fig = px.scatter_mapbox(date_data, 
#                             lat="Latitude", 
#                             lon="Longitude", 
#                             size="PM2.5 (µg/m³)", 
#                             color="PM2.5 (µg/m³)", 
#                             hover_name="City", 
#                             hover_data={"PM2.5 (µg/m³)": True, "Latitude": False, "Longitude": False},
#                             size_max=30,
#                             zoom=2,
#                             mapbox_style="open-street-map",
#                             title=f"PM2.5 on {selected_date}")
#     st.plotly_chart(fig)

#     # 선택된 날짜의 데이터 테이블
#     st.write(f"{selected_date}의 데이터:")
#     st.table(date_data[['City', 'PM2.5 (µg/m³)', 'PM10 (µg/m³)']])

# # 데이터 정보
# with st.expander("원본 데이터 미리보기"):
#     st.dataframe(data.head())
#     st.write(f"총 데이터 행 수: {len(data)}")


2025-03-17 20:23:04.475 No runtime found, using MemoryCacheStorageManager
2025-03-17 20:23:04.475 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-17 20:23:04.475 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-17 20:23:06.002 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-17 20:23:06.002 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-17 20:23:06.002 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-17 20:23:06.002 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-17 20:23:06.002 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-17 20:23:06.002 Thread 'MainThread':